<a href="https://colab.research.google.com/github/Zhekuson/Findan/blob/main/Sb_1/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Первоначальная настройка

Здесь маунт, импорты и прочее  
Не забудь очистить выходы ячеек перед коммитом

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls

In [7]:
!ls drive/MyDrive/Findan/Sb_1/

In [9]:
!git status

fatal: not a git repository (or any of the parent directories): .git


# Новый раздел